In [5]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sympy import *

In [47]:
g_c=32.2 #(lbm-ft)/(lbf-second^2)
v1_m3kg=1 # удельный объем м^3/кг
fg=0.1 # весовая доля газа в потоке
fo=0.5 # весовая доля нефти в потоке
fw=0.4 # весовая доля воды в потоке
M_m=29 # молекулярный вес, моль
z=0.999 # коэффициент сжимаемости газа
gamma_o=45 #плотность нефти API
F=1.4 #показатель адиабаты Cp/Cv
T=200 
Cvg=z*R_ftIbtlbmmolR/(M_m*(F-1)) # ???? теплоемкость при постоянном объеме для газа (ft-Ibf)/(lbm- OF)
Cvo=778*((0.355+0.00176*gamma_o)+(0.0051+1.167*gamma_o/100000)*T) #  теплоемкость при постоянном объеме для нефти (ft-Ibf)/(lbm- OF)
Cvw=4176 # ???? теплоемкость при постоянном объеме для воды (ft-Ibf)/(lbm- OF)
R_ftIbtlbmmolR=1 # ???? универсальная газовая постоянная  (ft-Ibt)/(lbm mol-R)
roo=1 # плотность нефти  Ibm/ft3
row=1 # плотность воды  Ibm/ft3
p2_psia=10
p1_atm=1
d1_mm=30 #диаметр трубы до штуцера
d2_mm=5 #диаметр штуцера
def wi_lbmsec(p1_atm, v1_m3kg):
    p1_psia=p1_atm*14.2233
    d1_ft=d1_mm/304.8
    d2_ft=d2_mm/304.8
    v1_ft3lbm=v1_m3kg*16.01845
    alf=(1/v1_ft3lbm)*(fo/roo+fw/row)
    pr=p2_psia/p1_psia
    A1_ft2=3.14*d1_ft**2/4
    A2_ft2=3.14*d2_ft**2/4
    n=(fg*F*Cvg+fo*Cvo+fw*Cvw)/(fg*Cvg+fo*Cvo+fw*Cvw)
    lambd=fg+((fg*Cvg+fo*Cvo+fw*Cvw)*M_m/(z*R_ftIbtlbmmolR))
    Ab=(lambd*(1-pr**((n-1)/n))+alf*(1-pr))/((1-((A2/A1)**2)*((fg+alf)/((fg*pr**(-1/n))+alf))**2)*((fg*pr**(-1/n))+alf)**2)
    wi_lbmsec=A2*((288*g_c*p1_psia/v1_ft3lbm)*Ab)**0.5
    return wi_lbmsec
print(wi_lbmsec(p1_atm, v1_m3kg))

168.948190083513
